# Installing Packages

In [1]:
# !pip install ultralytics


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


# Importing Packages

In [3]:
from ultralytics import YOLO
import os
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt

# Loading a pre-trained Yolov8 model

In [4]:
# Load a model
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

100%|██████████| 6.25M/6.25M [00:00<00:00, 30.8MB/s]


# Training the Model

In [ ]:
# Train the model
results = model.train(data="data.yaml",
                        epochs=100,
                        imgsz=416,
                        patience=10,
                        batch=32,
                        save=True,
                        cache=False, #if batch size == 32, 32gb ram crashes
                        project="runs_v8n/train",
                        pretrained=True,
                        verbose=True,
                        val=True,)

# Validating the fine-tuned model on the valid set

In [ ]:
# metrics = model.val()  # evaluate model performance on the validation set

# Testing on some random examples of the test set

### The below cell, has the entire actual 18 classes, just to get dictionary of labels from it

In [5]:
Data_Classes = ['Aluminium foil', 'Bottle cap', 'Bottle', 'Broken glass', 'Can', 'Carton', 'Cigarette', 'Cup', 'Lid', 'Other litter', 'Other plastic', 'Paper', 'Plastic bag - wrapper', 'Plastic container', 'Pop tab', 'Straw', 'Styrofoam piece', 'Unlabeled litter']

# Creating label2idx dictionary
Actual_label2idx = {label: idx for idx, label in enumerate(Data_Classes)}

# Creating idx2label dictionary
Actual_idx2label = {idx: label for idx, label in enumerate(Data_Classes)}

Actual_label2idx, Actual_idx2label

({'Aluminium foil': 0,
  'Bottle cap': 1,
  'Bottle': 2,
  'Broken glass': 3,
  'Can': 4,
  'Carton': 5,
  'Cigarette': 6,
  'Cup': 7,
  'Lid': 8,
  'Other litter': 9,
  'Other plastic': 10,
  'Paper': 11,
  'Plastic bag - wrapper': 12,
  'Plastic container': 13,
  'Pop tab': 14,
  'Straw': 15,
  'Styrofoam piece': 16,
  'Unlabeled litter': 17},
 {0: 'Aluminium foil',
  1: 'Bottle cap',
  2: 'Bottle',
  3: 'Broken glass',
  4: 'Can',
  5: 'Carton',
  6: 'Cigarette',
  7: 'Cup',
  8: 'Lid',
  9: 'Other litter',
  10: 'Other plastic',
  11: 'Paper',
  12: 'Plastic bag - wrapper',
  13: 'Plastic container',
  14: 'Pop tab',
  15: 'Straw',
  16: 'Styrofoam piece',
  17: 'Unlabeled litter'})

### Below cell has the custom filtered dataset labels

In [6]:
filter_list = [
    'Bottle cap',
    'Bottle',
    'Can',
    'Carton',
    'Cup',
    'Paper',
    'Straw',
    'Unlabeled litter',
]

### In the cell below
- We take some random examples from the test set
- Get predictions on the images
- Display the predicted image
- Draw the actual labels (only those labels that we kept in the finetuning process)
- Display the actual image (Note: you may see difference in colors of the Bounding boxes, because yolo uses random colors everytime)
- Subplot of both the images size by side
- Saved image in the folder = Local_Testing

In [7]:
Path_Test_Set = './V1/test'

Images_Path = os.path.join(Path_Test_Set, 'images')
Labels_Path = os.path.join(Path_Test_Set, 'labels')

Random_Images = os.listdir(Images_Path)
Random_Labels = os.listdir(Labels_Path)

Random_Images = np.random.choice(Random_Images, 5)
Random_Labels = [filename[:-3] + 'txt' for filename in Random_Images]

# Set the font size
font_size = 30  # Adjust as needed
try:
    font = ImageFont.truetype("arial.ttf", font_size)
except IOError:
    font = ImageFont.load_default(size=font_size)

# Loop over the selected images and labels
for image, label in zip(Random_Images, Random_Labels):
    results = model.predict(source=os.path.join(Images_Path, image))
    for result in results:
        boxes = result.boxes  # Boxes object for bounding box outputs
        masks = result.masks  # Masks object for segmentation masks outputs
        probs = result.probs  # Class probabilities for classification outputs
    
    # Save the predicted results
    os.makedirs('Local_Testing_v8n', exist_ok=True)
    pred_img_path = os.path.join('Local_Testing_v8n', label[:-3] + 'jpg')
    results[0].save(filename=pred_img_path)

    # Now plot the actual labels
    with open(os.path.join(Labels_Path, label)) as f:
        lines = f.readlines()
    
    content_copy = lines.copy()
    for line in lines:
        label_id = int(line.split(" ")[0])
        if Actual_idx2label[label_id] not in filter_list:
            content_copy.remove(line)
    
    # Open the image to draw bounding boxes
    img = Image.open(os.path.join(Images_Path, image))
    draw = ImageDraw.Draw(img)

    # Draw the filtered bounding boxes on the image
    for line in content_copy:
        label_id, x_center, y_center, width, height = map(float, line.split())
        
        # Convert from relative to absolute coordinates
        img_width, img_height = img.size
        x_min = (x_center - width / 2) * img_width
        x_max = (x_center + width / 2) * img_width
        y_min = (y_center - height / 2) * img_height
        y_max = (y_center + height / 2) * img_height
        
        # Draw the bounding box
        draw.rectangle([x_min, y_min, x_max, y_max], outline="red", width=3)

        # Write text
        draw.text((x_min, y_min), Actual_idx2label[int(label_id)], font=font, fill="red")
    
    # Save the image with drawn bounding boxes
    actual_img_path = os.path.join('Local_Testing_v8n', 'Actual_' + image)
    img.save(actual_img_path)
    
    # Create a subplot with the predicted and actual images side by side
    pred_img = Image.open(pred_img_path)
    actual_img = Image.open(actual_img_path)
    
    fig, axs = plt.subplots(1, 2, figsize=(15, 10))
    
    axs[0].imshow(pred_img)
    axs[0].set_title('Predicted')
    axs[0].axis('off')
    
    axs[1].imshow(actual_img)
    axs[1].set_title('Actual')
    axs[1].axis('off')
    
    plt.tight_layout()
    
    # Save the combined plot
    combined_img_path = os.path.join('Local_Testing_v8n', 'Combined_' + image)
    plt.savefig(combined_img_path)
    plt.close()


image 1/1 /Users/rehmansarwar/Downloads/final_code/V1/test/images/000020_JPG_jpg.rf.ce797f2c6b83b0b782eca9948df6def1.jpg: 640x640 (no detections), 154.0ms
Speed: 5.5ms preprocess, 154.0ms inference, 13.3ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /Users/rehmansarwar/Downloads/final_code/V1/test/images/000002_JPG_jpg.rf.3b3330b9601052ca754674984f0b6da4.jpg: 640x640 1 cat, 1 book, 95.3ms
Speed: 2.2ms preprocess, 95.3ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /Users/rehmansarwar/Downloads/final_code/V1/test/images/000012_JPG_jpg.rf.4dbfff1565881144195a6859c139f37b.jpg: 640x640 1 bird, 78.4ms
Speed: 2.3ms preprocess, 78.4ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /Users/rehmansarwar/Downloads/final_code/V1/test/images/000011_jpg.rf.72bbc41da6ff87f886d8275520a5a359.jpg: 640x640 (no detections), 95.6ms
Speed: 3.0ms preprocess, 95.6ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

ima

In [ ]:
from ultralytics import YOLO
model = YOLO("./runs_v8n/train/train/weights/best.pt")
path = model.export(format="onnx")